# rain_cal.py Program and Uses

Here is the paper that was relevent to the development of the code:
Automatic Identification of Rainfall in Acoustic Recordings by 
Carol Bedoya et al. - https://www.researchgate.net/publication/312324083_Automatic_identification_of_rainfall_in_acoustic_recordings

Summary of Automatic Identification of Rainfall in Acoustic Recordings: Normally, rainfall would be like background noise, making it very hard to detect on a spectrogram. However, the power spectral density shows the energy (power) at a given frequency in a different format that allows us to notice whether the energy is weak or strong. On the 600-1200 Hz frequencies, there is a higher energy reading when there is rain, meaning we can interpret spikes in the 600-1200 Hz frequency as rain. However, it's still not enough to just use the power spectral density, as other animals or anthropogenic sources also emit sounds on that 600-1200 frequency. Thus, the signal to noise ratio is used to determine whether the sound is rainfall or something else. The signal to noise ratio is basically the ratio between a wanted sound to noise - the ratio was determined using complicated math you can read about in the paper. 

Below is our code we developed based off of the paper. While the code here is similar to rain_cal.py, there are some differences, but not very noticable. While it was developed as a rain detection system, it struggles with differentiating between rain and wind. However, it also works well as an anthropogenic noise detection system after we set the t_mean value to 1.5-2 (we will show how to modify the code to detect anthropogenic noises). It also works well to find any sound files that have too much electrostatic interference. 

Please import the following libraries below:

In [1]:
from __future__ import division
import sys
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
from scipy import signal
from scipy.io import wavfile
import scipy.signal as scipy_signal

# Versions of Libraries that were used (most recent versions should work, but just in case):

future == 0.18.2

sys == 

numpy == 1.18.5

matplotlib == 3.2.1

glob == 

time == 

scipy == 1.4.1

scipy.io ==

scipy.signal == 

# Functions

In [2]:
"""
Calculate thresholds for rain detection in audio files
:param data_path: path to known rain recordings
:return: min threshold for mean PSD, max threshold for mean PSD, 
		 min threshold for SNR, max threshold for snr
"""
def train(data_path):
	sample_rate = None
	recording = []
	psd_arr = []
	snr_arr = []
	for file in glob.glob(data_path + '*.wav'):
		# load wav data
		try:
			rate, data = wavfile.read(file)
		except Exception as e:
			print('(failed) ' + file)
			print('\t' + str(e))
			continue

		recording = np.asarray(data)
		sample_rate = rate
		# import pdb; pdb.set_trace()
		length = recording.shape[0] / sample_rate
		# print(file)
		# print('sample rate = %d' % sample_rate)
		# print('length = %.1fs' % length)

		# import pdb;pdb.set_trace()
		# Stereo to mono
		if recording.ndim == 2:
			recording = recording.sum(axis=1) / 2

		# Downsample to 44.1 kHz
		if sample_rate != 44100:
			recording = signal.decimate(recording, int(sample_rate/44100))
			sample_rate = 44100

		# STEP 1: Estimate PSD vector from signal vector
		f, p = signal.welch(recording, fs=sample_rate, window='hamming', 
							nperseg=512, detrend=False)
		p = np.log10(p)

		# STEP 2: Extract vector a (freq band where rain lies) from PSD vector
		# min and max freq of rainfall freq band
		# divide by sample_rate to normalize from 0 to 1
		rain_min = (2.0 * 600) / sample_rate
		rain_max = (2.0 * 1200) / sample_rate

		limite_inf = int(round(p.__len__() * rain_min))
		limite_sup = int(round(p.__len__() * rain_max))

		# section of interest of the power spectral density
		a = p[limite_inf:limite_sup]
		# print(limite_inf)
		# print(limite_sup)
		# print(a)

		# STEP 3: Compute c (SNR of the PSD in rain freq band)
		# upper part of algorithm 2.1
		mean_a = np.mean(a)
		# lower part of algorithm 2.1
		std_a = np.std(a)

		# snr
		c = mean_a / std_a

		psd_arr.append(mean_a)
		snr_arr.append(c)

	return min(psd_arr), max(psd_arr), min(snr_arr), max(snr_arr)


"""
Classify clips as rain or non-rain
:param data_path: path to recordings
:param min_psd: minimum threshold for the mean value of the PSD
:param max_psd: maximum threshold for the mean value of the PSD
:param min_snr: Minimum threshold for the signal to noise ratio
:param max_snr: Maximum threshold for the signal to noise ratio
:return: indicator for rainy clips, metric of intensity of rainfall
"""
def classify(data_path, min_psd, max_psd, min_snr, max_snr):
	sample_rate = None
	recording = []
	for file in glob.glob(data_path + '*.wav'):
		# load wav data     
		try:
			rate, data = wavfile.read(file)
		except Exception as e:
			print('(failed) ' + file)
			print('\t' + str(e))
			continue

		recording = np.asarray(data)
		sample_rate = rate
		# import pdb; pdb.set_trace()
		length = recording.shape[0] / sample_rate
		# print(file)
		# print('sample rate = %d' % sample_rate)
		# print('length = %.1fs' % length)

		# import pdb;pdb.set_trace()
		# Stereo to mono
		if recording.ndim == 2:
			recording = recording.sum(axis=1) / 2

		downsample_rate = 44100
		downsample_ratio = int(sample_rate / downsample_rate)
		# Downsample to chosen rate
		if sample_rate != downsample_rate:
			recording = signal.decimate(recording, downsample_ratio)
			# recording = samplerate.resample(recording, downsample_ratio, "sinc_fastest")
			#recording = scipy_signal.resample(recording, 
					#int(len(recording)*downsample_ratio))
			# recording = resampy.resample(np.float64(recording),
			# 		sample_rate, downsample_rate)
			sample_rate = downsample_rate

		# STEP 1: Estimate PSD vector from signal vector
		f, p = signal.welch(recording, fs=sample_rate, window='hamming', 
							nperseg=512, detrend=False)
		p = np.log10(p)

		# STEP 2: Extract vector a (freq band where rain lies) from PSD vector
		# min and max freq of rainfall freq band
		# divide by sample_rate to normalize from 0 to 1
		rain_min = (2.0 * 600) / sample_rate
		rain_max = (2.0 * 1200) / sample_rate

		limite_inf = int(round(p.__len__() * rain_min))
		limite_sup = int(round(p.__len__() * rain_max))

		# section of interest of the power spectral density
		a = p[limite_inf:limite_sup]
		# print(limite_inf)
		# print(limite_sup)
		# print(a)

		# STEP 3: Compute c (SNR of the PSD in rain freq band)
		# upper part of algorithm 2.1
		mean_a = np.mean(a)
		# lower part of algorithm 2.1
		std_a = np.std(a)

		# snr
		c = mean_a / std_a

		# STEP 4: Classify samples
		if (min_psd <= mean_a <= max_psd) and (min_snr <= c <= max_snr):
			print('{}: Rainfall of intensity {:.2f}'.format(file, mean_a))

# Files Included on Github

20190623_150000 - Rain and wind

20190623_154000 - Rain and wind

20190611_100000 - People talking with a motor in the background

20190618_113000 - Electrostatic noises interfering with cricket chirps

20190623_013000 - Car engine momentarily passing through with sputters caught

# Rain Detection System

Struggles with differentiating between rain and wind. But it may still be of use. It functions on the 600-1200 Hz frequency. Basically, it will filter out anything that has an intensity higher than 2. Usually, that will be anthropogenic noises or animals. 

In [3]:
if __name__ == '__main__':
	### Set True or False depending on whether you want to train or use default thresholds ###
	TRAIN = False

	### Set your own file path ###
	train_path = 'E:/Internships/Acoustic Species/Mixed_AM_Datasets/Mixed_AM_Dataset/'
	classify_path = 'Mixed_AM_Datasets/'
    
	if TRAIN:
		start_time = time.time()
		min_psd, max_psd, min_snr, max_snr = train(train_path)
		print('----- Training: {:.2f} seconds -----'.format(
				time.time() - start_time))
	else:
		min_psd, max_psd, min_snr, max_snr = 1e-6, 2, 3.5, sys.maxsize

	start_time = time.time()
	classify(classify_path, min_psd, max_psd, min_snr, max_snr)
	print('----- Classification: {:.2f} seconds -----'.format(
			time.time() - start_time))

Mixed_AM_Datasets\20190623_150000.WAV: Rainfall of intensity 1.20
Mixed_AM_Datasets\20190623_154000.WAV: Rainfall of intensity 1.69
----- Classification: 20.08 seconds -----


# Anthropogenic Noises/Electrostatic Noises

In this, we kept listening to a frequency of 600-1200 Hz, but changed the threshold of the minimum PSD to 2. That way, we can detect human noises. It can also detect electrostatic noises, as seen with one clip caught. 

In [4]:
if __name__ == '__main__':
	### Set True or False depending on whether you want to train or use default thresholds ###
	TRAIN = False

	### Set your own file path ###
	train_path = 'E:/Internships/Acoustic Species/Mixed_AM_Datasets/Mixed_AM_Dataset/'
	classify_path = 'Mixed_AM_Datasets/'

	if TRAIN:
		start_time = time.time()
		min_psd, max_psd, min_snr, max_snr = train(train_path)
		print('----- Training: {:.2f} seconds -----'.format(
				time.time() - start_time))
	else:
		min_psd, max_psd, min_snr, max_snr = 2, sys.maxsize, 3.5, sys.maxsize

	start_time = time.time()
	classify(classify_path, min_psd, max_psd, min_snr, max_snr)
	print('----- Classification: {:.2f} seconds -----'.format(
			time.time() - start_time))


Mixed_AM_Datasets\20190611_100000.WAV: Rainfall of intensity 3.54
Mixed_AM_Datasets\20190618_113000.WAV: Rainfall of intensity 2.28
Mixed_AM_Datasets\20190623_013000.WAV: Rainfall of intensity 3.61
----- Classification: 18.79 seconds -----
